In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
import tokenization

## <center>NLP Disaster Tweet Classification w/ roBERTa</center>

This notebook implements a roBERTa model in Tensorflow to evaluate whether a tweet is about a disaster or not. I have provided explanations throughout to provide a better understanding of what the roBERTa model is actually doing.

I got most of my understanding for this notebook from a good discussion thread about the roBERTa model from @Chris Deotte explaining how the components of the model work and his starter notebook on the roBERTa model. These are the top two links below. I also found the Tensorflow documentation quite informative as well (third and fourth links).

### Useful Links

This is a collection of links that I found helpful in understanding the structure of the roBERTa model, how it works, and more.

- [TensorFlow roBERTa Explained Discussion](https://www.kaggle.com/c/tweet-sentiment-extraction/discussion/143281#807401)
- [tensorflow-roberta-0-705 Notebook](https://www.kaggle.com/cdeotte/tensorflow-roberta-0-705)
- [Bert_en_uncased Docs](https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4)
- [bert_en_uncased_preprocess](https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3)
- [How to get meaning from text with language model BERT](https://www.youtube.com/watch?v=-9vVhYEXeyQ)
- [TF Bert Tokenizer](https://github.com/google-research/bert)

In [2]:
#setting a seed for reproducability
SEED = 1002
def seed_everything(seed):
    np.random.seed(seed)
    tf.random.set_seed(seed) 
    
seed_everything(SEED) 

Reading in the data using pandas. We will tokenize the text later in the notebook.

In [3]:
#reading input data with pandas
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

#visualizing some of the tweets
for i, val in enumerate(train.iloc[:2]["text"].to_list()):
    print("Tweet {}: {}".format(i+1, val))

Tweet 1: Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
Tweet 2: Forest fire near La Ronge Sask. Canada


## <center>bert_encode function</center>


### tokenizer
We are using the [Tensorflow Research's BERT tokenization method](https://github.com/tensorflow/models/blob/master/official/nlp/bert/tokenization.py). This tokenization method can be thought of as three steps.

- Text Normalization
    - The first part of the tokenizer converts the text to lowercase (given that we are using the uncased version of roBERTa), converts whitespace to spaces, and strips out accent markers.
    ```
    "Alex Pättason's, "  -> "alex pattason's,"
    ```
    <br></br>
- Punctuation splitting
    - This next step adds spaces on each side of all "punctuation". Note that this includes any non-letter/number/space ASCII characters (ie including \$, \@). See more of this in the Docs. 
    ```
    "Alex Pättason's, "  -> "alex pattason ' s ,"
    ```
    <br></br>
- WordPiece tokenization
    - This step applies what is called whitespace tokenization to the output of the process above, and apply's WordPiece tokenization to each word separately. See the example below.
    ```
    "Alex Pättason's, "  -> "alex pat ##ta ##son ' s ,"
    ```
   
### tags
The next part of the function reduces the length of the text by the max_length that we have specified and adds [CLS] and [SEP] tags to the end of the array. The [CLS] tag is short for classification and indicates the start of the sentence. Similarly, the [SEP] tag indicates the end of the sentence.

### convert_tokens_to_ids + pad_masks

We then use the tokenizer method to replace the string representation of words with integers. We also create the input mask (AKA pad_masks), and the segment id's. Note that we are not fulfilling the segment_ids full benefits below as we are only passing an array of zeros. More on the tokens, pad_masks, and segment_ids further in the notebook.

In [4]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

## <center>build_model function</center>

The first three components of the function are basically preprocessing plain text inputs into the input format expected by the roBERTa model.

### input_word_ids
- Basically maps each word to its token id. There can be multiple different values that correspond with the same word. For example, "smell" could be encoded both as 883 and 789.
<br></br>
```
text = "I love this notebook. It is Great."
input_word_ids = [10, 235, 123, 938, 184, 301, 567]
```

### the input_mask
- Shows where the sentence begins, and where it ends using an array. All input tokens that are not padding are given a value of 1, and all values that are padding are given 0. If the sentence exceeds that max_length, then the entire vector will be of 1's.
<br></br>
```
text = "I love this notebook. It is Great."
input_mask = [1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
```

### segment_ids
- This component is still a little vague for me, but from my understanding, it is recognizing segments of the text. The start of each segment has a 1 in the array, and other components and padding all have a zero. I am unsure as to whether this corresponds to the end of sentences or paragraphs, but if you can explain this better please do so in the comments below!
<br></br>
```
text = "I love this notebook. It is Great."
segment_ids = [1, 0, 0, 0, 1, 0, 0, 0, 0, 0]
```

In [5]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    #could be pooled_output, sequence_output yet sequence output provides for each input token (in context)
    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    
    #specifying optimizer
    model.compile(Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

## Build Model + Preprocess Data

The first cell below is basically loading in the version of the roBERTa model that we want to use. We are using a Large uncased model. The most simple way to use a roBERTa model and modify it to a specific use case is to set it as a KerasLayer.

Note there are many different variations of BERT models that you can look through here --> [TFhub Bert](https://tfhub.dev/google/collections/bert/1)


In [6]:
#load uncased bert model
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

2021-12-02 23:09:43.873392: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-02 23:09:43.968548: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-02 23:09:43.969256: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-02 23:09:43.970634: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In the next cell, we are setting up the tokenizer that will be used to preprocess our input data to what BERT understands. We have to specify a vocab file so that the tokenizer knows what number to encode each word as then we have to specify whether we want uncased or cased text. We will use the same vocab_file that the pre-trained model was trained on (Google's SentencePiece in this case) and we will also use the same case that the model was built for (uncased).

Finally, once we have these two variables, we create the tokenizer and tokenize the training and testing data using the bert_encode function that we created above. 

In [7]:
#vocab file from pre-trained BERT for tokenization
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()

#returns true/false depending on if we selected cased/uncased bert layer
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

#Create the tokenizer
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

#tokenizing the training and testing data
train_input = bert_encode(train.text.values, tokenizer, max_len=160)
test_input = bert_encode(test.text.values, tokenizer, max_len=160)
train_labels = train.target.values

Having a look at the model summary. We can see the three input layers that we created followed by the roBERTa model which is in the keras_layer. We have the final dense layer which predicts the sentiment of the tweet on a scale of 0-1. 

In [8]:
model = build_model(bert_layer, max_len=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

## Training Model

The next cell is a simple way of training the model using Keras. We have included the built-in ModelCheckpoint callback to only save the model that has the highest validation accuracy. This ensures we are only saving the best models. 


We could decrease the randomness of the split by doing some sort of a stratified split, or cross-validation, but this will do for now.

In [9]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True)

train_history = model.fit(
    train_input, train_labels,
    validation_split=0.1,
    epochs=3,
    callbacks=[checkpoint],
    batch_size=16
)

Epoch 1/3


2021-12-02 23:10:07.638823: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


429/429 [==============================] - 440s 952ms/step - loss: 0.4205 - accuracy: 0.8213 - val_loss: 0.3730 - val_accuracy: 0.8465
Epoch 2/3
429/429 [==============================] - 407s 948ms/step - loss: 0.2719 - accuracy: 0.8948 - val_loss: 0.4138 - val_accuracy: 0.8596
Epoch 3/3
429/429 [==============================] - 407s 949ms/step - loss: 0.1518 - accuracy: 0.9425 - val_loss: 0.5273 - val_accuracy: 0.8491


## Make Prediction

Using the model to make predictions on the testing set. We round the prediction to 1 or 0. 1 is a disaster tweet, and 0 is a regular tweet.

In [10]:
test_pred = model.predict(test_input)

submission['target'] = test_pred.round().astype(int)
submission.to_csv('submission.csv', index=False)